In [1]:
import pandas as pd
import numpy as np

In [2]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
retail_dataframe = pd.read_excel("Online Retail.xlsx")
retail_dataframe.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
germany_retail_df = retail_dataframe[retail_dataframe['Country']=='Germany']
germany_retail_df['Description'].fillna('Desc Missing', inplace=True)
germany_retail_df['CustomerID']=germany_retail_df['CustomerID'].apply(np.int64)

//anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
german_cancelled_trans_df = germany_retail_df[germany_retail_df['InvoiceNo'].astype('str').str.startswith('C', na=False)]
german_sale_trans_df = germany_retail_df[~germany_retail_df['InvoiceNo'].isin(german_cancelled_trans_df.InvoiceNo)]

In [38]:
german_sale_trans_df.shape[0]

9042

# Create Training and Test sets

Training Data - Dec 1st 2010 to Oct 1st 2011

Test Data - Oct 2nd 2011 to Dec 10th 2011

In [7]:
train_df = german_sale_trans_df[(german_sale_trans_df['InvoiceDate'] > '2010-12-01') & (german_sale_trans_df['InvoiceDate'] <= '2011-10-01')]
test_df = german_sale_trans_df[(german_sale_trans_df['InvoiceDate'] > '2011-10-01') & (german_sale_trans_df['InvoiceDate'] <= '2011-12-10')]

In [38]:
print('Number of Train Data records- ',train_df.shape[0])
print('Number of Test Data records- ', test_df.shape[0])

Number of Train Data records-  6089
Number of Test Data records-  2579


Drop POSTAGE products as they dont add value in recommendations

In [8]:
train_df.drop(train_df[train_df['Description'] == 'POSTAGE'].index, axis=0, inplace=True)
test_df.drop(test_df[test_df['Description'] == 'POSTAGE'].index, axis=0, inplace=True)

//anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Training

# Using apriori

In [9]:
def encode_data(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [10]:
def create_basket(df):
    basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
    return basket 

In [11]:
bk = create_basket(train_df)
basket_sets = bk.applymap(encode_data)

Use support of 0.03, confidence of 65%

In [13]:
freq_items = apriori(basket_sets, min_support=0.03, use_colnames=True)
freq_items['length'] = freq_items['itemsets'].apply(lambda x:len(x))
freq_items.sort_values(by='support', ascending=False).head()

,support,itemsets,length
113,0.268608,(ROUND SNACK BOXES SET OF4 WOODLAND ),1
112,0.184466,(ROUND SNACK BOXES SET OF 4 FRUITS ),1
247,0.155340,"(ROUND SNACK BOXES SET OF 4 FRUITS , ROUND SNACK BOXES SET OF4 WOODLAND )",2
85,0.145631,(PLASTERS IN TIN WOODLAND ANIMALS),1
105,0.145631,(REGENCY CAKESTAND 3 TIER),1


In [14]:
asso= association_rules(freq_items, metric='confidence', min_threshold=.65).sort_values(by='lift', ascending=False)
asso.sort_values(by="lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
28,(SPACEBOY CHILDRENS CUP),(SPACEBOY CHILDRENS BOWL),0.042071,0.042071,0.038835,0.923077,21.940828,0.037065,12.453074
27,(SPACEBOY CHILDRENS BOWL),(SPACEBOY CHILDRENS CUP),0.042071,0.042071,0.038835,0.923077,21.940828,0.037065,12.453074
9,(CHILDRENS CUTLERY DOLLY GIRL ),(CHILDRENS CUTLERY SPACEBOY ),0.051780,0.051780,0.038835,0.750000,14.484375,0.036154,3.792880
10,(CHILDRENS CUTLERY SPACEBOY ),(CHILDRENS CUTLERY DOLLY GIRL ),0.051780,0.051780,0.038835,0.750000,14.484375,0.036154,3.792880
29,(SWEETHEART CERAMIC TRINKET BOX),(STRAWBERRY CERAMIC TRINKET BOX),0.045307,0.067961,0.042071,0.928571,13.663265,0.038992,13.048544
21,(RED STRIPE CERAMIC DRAWER KNOB),(WHITE SPOT RED CERAMIC DRAWER KNOB),0.045307,0.055016,0.032362,0.714286,12.983193,0.029870,3.307443
0,(BLUE HARMONICA IN BOX ),(RED HARMONICA IN BOX ),0.048544,0.051780,0.032362,0.666667,12.875000,0.029849,2.844660
26,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.067961,0.071197,0.058252,0.857143,12.038961,0.053414,6.501618
25,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.071197,0.067961,0.058252,0.818182,12.038961,0.053414,5.126214
4,(BLUE POLKADOT PLATE ),(BLUE POLKADOT CUP),0.042071,0.064725,0.032362,0.769231,11.884615,0.029639,4.052859


There is a 22 times more likihood to buy SPACEBOY CHILDRENS CUP than an avg customer if they but a SPACEBOY CHILDRENS BOWL. This rule is "true" in 92% of the cases (confidence). 
This can be used as insight to recommend SPACEBOY CHILDRENS CUP for those customers who bought SPACEBOY CHILDRENS BOWL.

Also, a customer is 14 times more likely to buy 'CHILDRENS CUTLERY SPACEBOY' than an average customer (lift) if he/she buys 'CHILDRENS CUTLERY DOLLY GIRL'. This rule is "true" in 75% of the cases (confidence). 
This can be used as insight to recommend CHILDRENS CUTLERY SPACEBOY for those customers who bought CHILDRENS CUTLERY DOLLY GIRL.

# Using APYORI package

In [3]:
pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5975 sha256=23c592d0da34ab78873de7c0d71d9b0495f902bb2dee07e57a5cab6c287f54d9
  Stored in directory: /Users/z060373/Library/Caches/pip/wheels/cb/f6/e1/57973c631d27efd1a2f375bd6a83b2a616c4021f24aab84080
Successfully built apyori
Note: you may need to restart the kernel to use updated packages.


In [15]:
from apyori import apriori

In [16]:
train=train_df.groupby('InvoiceNo')['Description'].apply(list).reset_index()
train_desc_list=train['Description'].tolist()
train_desc_list

[['SET OF 6 T-LIGHTS SANTA',
  'ROTATING SILVER ANGELS T-LIGHT HLDR',
  'MULTI COLOUR SILVER T-LIGHT HOLDER',
  '5 HOOK HANGER MAGIC TOADSTOOL',
  '3 HOOK HANGER MAGIC GARDEN',
  '5 HOOK HANGER RED MAGIC TOADSTOOL',
  'ASSORTED COLOUR LIZARD SUCTION HOOK',
  'JUMBO BAG WOODLAND ANIMALS',
  'JUMBO BAG OWLS',
  'HOT WATER BOTTLE BABUSHKA ',
  'HOMEMADE JAM SCENTED CANDLES',
  "CHILDREN'S CIRCUS PARADE MUG",
  'PACK 3 FIRE ENGINE/CAR PATCHES',
  'PICTURE DOMINOES'],
 ['JAM MAKING SET PRINTED',
  'JAM JAR WITH PINK LID',
  'JAM JAR WITH GREEN LID',
  'SWEETHEART CAKESTAND 3 TIER',
  '60 CAKE CASES VINTAGE CHRISTMAS',
  '6 RIBBONS RUSTIC CHARM',
  'CAKE STAND WHITE TWO TIER LACE',
  'PACK OF 60 PINK PAISLEY CAKE CASES',
  '60 TEATIME FAIRY CAKE CASES',
  'PACK OF 72 RETROSPOT CAKE CASES'],
 ['FELTCRAFT 6 FLOWER FRIENDS',
  '6 RIBBONS RUSTIC CHARM',
  'WOODLAND  HEIGHT CHART STICKERS ',
  'CHILDS BREAKFAST SET CIRCUS PARADE',
  'CHILDS BREAKFAST SET SPACEBOY ',
  'COFFEE MUG CAT + BIRD DESIG

In [17]:
german_trans_train = apriori(train_desc_list, min_support=0.03, min_confidence=0.65, min_lift=2, min_length=1)

In [18]:
results_train = list(german_trans_train)

In [19]:
values_train =[]

In [20]:
for item in results_train:
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    rows=(items[0],items[1],str(item[1]),str(item[2][0][2]),str(item[2][0][3]))  
    values_train.append(rows)

In [22]:
labels=['Title1', 'Title2','Support','Confidence','Lift']
products_sugg_train = pd.DataFrame(data=values_train, columns=labels)
products_sugg_train['Lift']=products_sugg_train['Lift'].astype(float)
products_sugg_train.sort_values(by='Lift', ascending=False).head(10)

,Title1,Title2,Support,Confidence,Lift
25,SPACEBOY CHILDRENS BOWL,SPACEBOY CHILDRENS CUP,0.038834951456310676,0.923076923076923,21.940828
9,CHILDRENS CUTLERY DOLLY GIRL,CHILDRENS CUTLERY SPACEBOY,0.038834951456310676,0.7499999999999999,14.484375
26,SWEETHEART CERAMIC TRINKET BOX,STRAWBERRY CERAMIC TRINKET BOX,0.042071197411003236,0.9285714285714285,13.663265
20,WHITE SPOT RED CERAMIC DRAWER KNOB,RED STRIPE CERAMIC DRAWER KNOB,0.032362459546925564,0.7142857142857142,12.983193
0,BLUE HARMONICA IN BOX,RED HARMONICA IN BOX,0.032362459546925564,0.6666666666666666,12.875000
24,SET/6 RED SPOTTY PAPER PLATES,SET/6 RED SPOTTY PAPER CUPS,0.05825242718446602,0.8571428571428572,12.038961
4,BLUE POLKADOT CUP,BLUE POLKADOT PLATE,0.032362459546925564,0.7692307692307692,11.884615
10,COFFEE MUG APPLES DESIGN,COFFEE MUG PEARS DESIGN,0.045307443365695796,0.875,11.755435
2,BLUE POLKADOT CUP,BLUE POLKADOT BOWL,0.032362459546925564,0.7142857142857142,11.035714
11,JAM JAR WITH PINK LID,JAM JAR WITH GREEN LID,0.045307443365695796,0.9333333333333335,10.681481


There is a 22 times more likihood to buy SPACEBOY CHILDRENS CUP than an avg customer if they but a SPACEBOY CHILDRENS BOWL. This rule is "true" in 92% of the cases (confidence). This can be used as insight to recommend SPACEBOY CHILDRENS CUP for those customers who bought SPACEBOY CHILDRENS BOWL.

Also, a customer is 14 times more likely to buy 'CHILDRENS CUTLERY SPACEBOY' than an average customer (lift) if he/she buys 'CHILDRENS CUTLERY DOLLY GIRL'. This rule is "true" in 75% of the cases (confidence). This can be used as insight to recommend CHILDRENS CUTLERY SPACEBOY for those customers who bought CHILDRENS CUTLERY DOLLY GIRL.

We can notice that the results are similar from both the packages

# TEST

In [34]:
test=test_df.groupby('InvoiceNo')['Description'].apply(list).reset_index()
test_desc_list=test['Description'].tolist()
test_desc_list

[['EMERGENCY FIRST AID TIN ',
  'TRAVEL SEWING KIT',
  'ROUND SNACK BOXES SET OF4 WOODLAND ',
  'PACK OF 12 LONDON TISSUES ',
  'VINTAGE DOILY TRAVEL SEWING KIT',
  'VINTAGE DOILY DELUXE SEWING KIT ',
  'SET 12 COLOURING PENCILS DOILY',
  'SET 12 COLOUR PENCILS LOVE LONDON',
  'SET 12 COLOUR PENCILS SPACEBOY ',
  'RED RETROSPOT TAPE',
  "3 RAFFIA RIBBONS 50'S CHRISTMAS ",
  "WRAP 50'S  CHRISTMAS",
  'CHRISTMAS CRAFT LITTLE FRIENDS',
  'PACK 3 BOXES CHRISTMAS PANNETONE',
  "BISCUIT TIN 50'S CHRISTMAS",
  "GIANT 50'S CHRISTMAS CRACKER",
  "BOX OF 6 MINI 50'S CRACKERS",
  "PACK OF 12 50'S CHRISTMAS TISSUES",
  'POPCORN HOLDER',
  'PARTY CONE CHRISTMAS DECORATION ',
  'WOODLAND MINI BACKPACK',
  'LUNCH BAG  BLACK SKULL.',
  'LUNCH BAG VINTAGE DOILY ',
  'LUNCH BAG WOODLAND',
  'PACK OF 12 PAISLEY PARK TISSUES ',
  'SET/20 RED RETROSPOT PAPER NAPKINS ',
  'JAM JAR WITH PINK LID',
  'JAM JAR WITH GREEN LID',
  'RED RETROSPOT TISSUE BOX'],
 ['SET OF 6 SNACK LOAF BAKING CASES',
  'WRAP RED VIN

In [35]:
german_trans_test = apriori(test_desc_list, min_support=0.03, min_confidence=0.65, min_lift=2, min_length=1)

In [36]:
results_test = list(german_trans_test)
values_test =[]
for item in results_test:
    # first index of the inner list
    # Contains base item and add item
    pair = item[0] 
    items = [x for x in pair]
    rows=(items[0],items[1],str(item[1]),str(item[2][0][2]),str(item[2][0][3]))  
    values_test.append(rows)
    
labels=['Title1', 'Title2','Support','Confidence','Lift']
products_sugg_test = pd.DataFrame(data=values_test, columns=labels)
products_sugg_test['Lift']=products_sugg_test['Lift'].astype(float)
products_sugg_test.sort_values(by='Lift', ascending=False).head(10)

,Title1,Title2,Support,Confidence,Lift
11,DOLLY GIRL CHILDRENS BOWL,DOLLY GIRL CHILDRENS CUP,0.03571428571428571,0.8333333333333333,23.333333
4,CHILDRENS CUTLERY DOLLY GIRL,CHILDRENS CUTLERY SPACEBOY,0.04285714285714286,0.8571428571428571,20.000000
32,CHILDRENS CUTLERY DOLLY GIRL,WOODLAND BUNNIES LOLLY MAKERS,0.03571428571428571,0.7142857142857142,20.000000
31,REGENCY CAKESTAND 3 TIER,CHILDRENS CUTLERY DOLLY GIRL,0.03571428571428571,0.7142857142857142,20.000000
29,WOODLAND SMALL BLUE FELT HEART,WOODLAND SMALL PINK FELT HEART,0.03571428571428571,0.8333333333333333,19.444444
30,WOODLAND SMALL PINK FELT HEART,WOODLAND SMALL RED FELT HEART,0.03571428571428571,0.8333333333333333,19.444444
2,PINK OWL SOFT TOY,BLUE OWL SOFT TOY,0.03571428571428571,1.0,17.500000
25,SKETCHBOOK MAGNETIC SHOPPING LIST,SET OF 6 SNACK LOAF BAKING CASES,0.03571428571428571,1.0,17.500000
27,SPACEBOY CHILDRENS BOWL,SPACEBOY CHILDRENS CUP,0.03571428571428571,0.8333333333333333,16.666667
10,CHOCOLATE BOX RIBBONS,SCANDINAVIAN REDS RIBBONS,0.03571428571428571,1.0,15.555556


From the test results, it can be seen the association between two products SPACEBOY CHILDRENS CUP and SPACEBOY CHILDRENS BOWL which had a highest lift in train data have a good lift in test data as well.
Similarly, 
It can be seen the association between two products CHILDRENS CUTLERY SPACEBOY and CHILDRENS CUTLERY DOLLY GIRL which had a good lift in train data have a good lift in test data as well. 

So we can surely stock these two products together as there is a high liklihood of these two products being purchased together. 

Observations -
There is a 17 times more likihood to buy SPACEBOY CHILDRENS CUP than an avg customer if they but a SPACEBOY CHILDRENS BOWL. This rule is "true" in 83% of the cases (confidence). This can be used as insight to recommend SPACEBOY CHILDRENS CUP for those customers who bought SPACEBOY CHILDRENS BOWL.

Also, a customer is 20 times more likely to buy 'CHILDRENS CUTLERY SPACEBOY' than an average customer (lift) if he/she buys 'CHILDRENS CUTLERY DOLLY GIRL'. This rule is "true" in 85% of the cases (confidence). This can be used as insight to recommend CHILDRENS CUTLERY SPACEBOY for those customers who bought CHILDRENS CUTLERY DOLLY GIRL.